In [1]:
import numpy as np
from matplotlib import pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval, Params
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from collections import defaultdict
import random
import pickle

import supervisely as sly
import supervisely.nn.benchmark

In [2]:
import supervisely
from supervisely.nn import benchmark

<module 'supervisely.nn.benchmark' from '/root/supervisely/supervisely/nn/benchmark/__init__.py'>

## Loading data

In [12]:
import supervisely.nn.benchmark
import supervisely.nn.benchmark.evaluation
import supervisely.nn.benchmark.evaluation.coco
import json

eval_dir_1 = "data/41774_COCO-100 (det)/66127_Serve YOLOv8 | v9 | v10/evaluation"
eval_dir_2 = "data/41774_COCO-100 (det)/66127_Serve YOLOv8 | v9 | v10_001/evaluation"


def read_eval_data(eval_dir):
    cocoGt_path, cocoDt_path, eval_data_path, inference_info_path = (
        eval_dir + "/cocoGt.json",
        eval_dir + "/cocoDt.json",
        eval_dir + "/eval_data.pkl",
        eval_dir + "/inference_info.json",
    )

    with open(cocoGt_path, "r") as f:
        cocoGt_dataset = json.load(f)
    with open(cocoDt_path, "r") as f:
        cocoDt_dataset = json.load(f)

    with open(eval_data_path, "rb") as f:
        eval_data = pickle.load(f)

    cocoGt = COCO()
    cocoGt.dataset = cocoGt_dataset
    cocoGt.createIndex()
    cocoDt = cocoGt.loadRes(cocoDt_dataset["annotations"])

    mp = supervisely.nn.benchmark.evaluation.coco.MetricProvider(
        eval_data['matches'], eval_data['coco_metrics'], eval_data['params'], cocoGt, cocoDt
    )
    return mp

mp1 = read_eval_data(eval_dir_1)
mp2 = read_eval_data(eval_dir_2)
mp1.calculate()
mp2.calculate()

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!


/root/supervisely/supervisely/nn/benchmark/evaluation/coco/metric_provider.py:257: RuntimeWarning: invalid value encountered in divide
  precision = tp / (tp + fp)
/root/supervisely/supervisely/nn/benchmark/evaluation/coco/metric_provider.py:279: RuntimeWarning: invalid value encountered in divide
  pr = tp / (tp + fp)
/root/supervisely/supervisely/nn/benchmark/evaluation/coco/metric_provider.py:281: RuntimeWarning: invalid value encountered in divide
  f1 = 2 * pr * rc / (pr + rc)
/root/supervisely/supervisely/nn/benchmark/evaluation/coco/metric_provider.py:257: RuntimeWarning: invalid value encountered in divide
  precision = tp / (tp + fp)
/root/supervisely/supervisely/nn/benchmark/evaluation/coco/metric_provider.py:279: RuntimeWarning: invalid value encountered in divide
  pr = tp / (tp + fp)
/root/supervisely/supervisely/nn/benchmark/evaluation/coco/metric_provider.py:281: RuntimeWarning: invalid value encountered in divide
  f1 = 2 * pr * rc / (pr + rc)


In [114]:
from pycocotools import mask as maskUtils

iouType = 'bbox'
iouThr = 0.5

key_name = 'bbox' if iouType == 'bbox' else 'segmentation'

imgId2ann1 = defaultdict(list)
imgId2ann2 = defaultdict(list)
for m in mp1.m.fp_matches:
    ann = mp1.cocoDt.anns[m['dt_id']]
    imgId2ann1[m['image_id']].append(ann)
for m in mp2.m.fp_matches:
    ann = mp2.cocoDt.anns[m['dt_id']]
    imgId2ann2[m['image_id']].append(ann)

same_fp_matches = []
for img_id in imgId2ann1:
    anns1 = imgId2ann1[img_id]
    anns2 = imgId2ann2[img_id]
    geoms1 = [x[key_name] for x in anns1]
    geoms2 = [x[key_name] for x in anns2]

    ious = maskUtils.iou(geoms1, geoms2, [0]*len(geoms2))
    if len(ious) == 0:
        continue
    indxs = np.nonzero(ious > iouThr)
    if len(indxs[0]) == 0:
        continue
    for i,j in zip(*indxs):
        same_fp_matches.append((anns1[i], anns2[j], ious[i,j]))

In [118]:
mp1.m.fn_matches[0]

{'image_id': 83,
 'category_id': 11,
 'dt_id': None,
 'gt_id': 725,
 'type': 'FN',
 'score': None,
 'iou': None,
 'miss_cls': False}

In [126]:
id1 = set([x['gt_id'] for x in mp1.m.fn_matches])
id2 = set([x['gt_id'] for x in mp2.m.fn_matches])

same_fn_matches = id1 & id2
diff_fn_matches_1 = id1 - id2
diff_fn_matches_2 = id2 - id1



In [128]:
len(same_fn_matches), len(diff_fn_matches_1), len(diff_fn_matches_2)

(383, 79, 6)

In [132]:
len(id1), len(id2)

(462, 389)

In [ ]:
cocoGt_path = "cocoGt_remap.json"
cocoDt1_path = "data/model-benchmark/COCO 2017 val (YOLOv8-L, conf-0.01)/cocoDt.json"
cocoDt2_path = "data/model-benchmark/COCO 2017 val (RT-DETR r34, conf-0.01)/cocoDt.json"
eval_data1_path = "eval_data_conf-0.01.pkl"
eval_data2_path = "eval_data_rtdetr_conf-0.01.pkl"

In [ ]:
cocoGt = COCO(cocoGt_path)
cocoDt1 = cocoGt.loadRes(cocoDt1_path)
cocoDt2 = cocoGt.loadRes(cocoDt2_path)

with open(eval_data1_path, 'rb') as f:
    eval_data1 = pickle.load(f)

with open(eval_data2_path, 'rb') as f:
    eval_data2 = pickle.load(f)

In [ ]:
from importlib import reload
reload(metric_provider)
m_full1 = metric_provider.MetricProvider(eval_data1['matches'], eval_data1['coco_metrics'], eval_data1['params'], cocoGt, cocoDt1)
m_full1.model_name = "YOLOv8-L"
m_full1.base_metrics()

In [ ]:
from importlib import reload
reload(metric_provider)
m_full2 = metric_provider.MetricProvider(eval_data2['matches'], eval_data2['coco_metrics'], eval_data2['params'], cocoGt, cocoDt2)
m_full2.model_name = "RT-DETR r34"
m_full2.base_metrics()

## F1-optimal conf

In [ ]:
score_profile = m_full1.confidence_score_profile()
# score_profile = m_full.confidence_score_profile_v0()
f1_optimal_conf1, best_f1 = m_full1.get_f1_optimal_conf()
print(f"F1-Optimal confidence: {f1_optimal_conf1:.4f} with f1: {best_f1:.4f}")

matches_thresholded = metric_provider.filter_by_conf(eval_data1['matches'], f1_optimal_conf1)
m1 = metric_provider.MetricProvider(matches_thresholded, eval_data1['coco_metrics'], eval_data1['params'], cocoGt, cocoDt1)
m1.model_name = "YOLOv8-L"
m1.base_metrics()

In [ ]:
score_profile = m_full2.confidence_score_profile()
# score_profile = m_full.confidence_score_profile_v0()
f1_optimal_conf2, best_f1 = m_full2.get_f1_optimal_conf()
print(f"F1-Optimal confidence: {f1_optimal_conf2:.4f} with f1: {best_f1:.4f}")

matches_thresholded = metric_provider.filter_by_conf(eval_data2['matches'], f1_optimal_conf2)
m2 = metric_provider.MetricProvider(matches_thresholded, eval_data2['coco_metrics'], eval_data2['params'], cocoGt, cocoDt2)
m2.model_name = "RT-DETR r34"
m2.base_metrics()

In [ ]:
model_metrics = [m1, m2]
model_metrics_full = [m_full1, m_full2]

## Overview

In [ ]:
# Overall Metrics comparison
base_metrics1 = m1.base_metrics()
base_metrics2 = m2.base_metrics()
r1 = list(base_metrics1.values())
r2 = list(base_metrics2.values())
theta = [metric_provider.METRIC_NAMES[k] for k in base_metrics1.keys()]
fig = go.Figure()
fig.add_trace(go.Scatterpolar(
    r=r1+[r1[0]], theta=theta+[theta[0]], name=m1.model_name,
    hovertemplate='%{theta}: %{r:.2f}<extra></extra>',
))
fig.add_trace(go.Scatterpolar(
    r=r2+[r2[0]], theta=theta+[theta[0]], name=m2.model_name,
    hovertemplate='%{theta}: %{r:.2f}<extra></extra>',
))
fig.update_layout(polar=dict(radialaxis=dict(range=[0., 1.]),
                             angularaxis=dict(rotation=90, direction='clockwise')),
                  title="Overall Metrics", width=600, height=500,)
fig.show()

In [ ]:
# Grouped bar chart
base_metrics1 = m1.base_metrics()
base_metrics2 = m2.base_metrics()
df = pd.DataFrame({
    "metric": [metric_provider.METRIC_NAMES[k] for k in base_metrics1.keys()],
    m1.model_name: list(base_metrics1.values()),
    m2.model_name: list(base_metrics2.values()),
})
fig = px.bar(df, x='metric', y=[m1.model_name, m2.model_name], barmode='group')
fig.update_layout(title="Overall Metrics")
fig.show()

## Outcome Counts

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=[m1.TP_count], y=[m1.model_name], name='TP', orientation='h', marker=dict(color='#1fb466')))
fig.add_trace(go.Bar(x=[m1.FN_count], y=[m1.model_name], name='FN', orientation='h', marker=dict(color='#dd3f3f')))
fig.add_trace(go.Bar(x=[m1.FP_count], y=[m1.model_name], name='FP', orientation='h', marker=dict(color='#d5a5a5')))

fig.add_trace(go.Bar(x=[m2.TP_count], y=[m2.model_name], name='TP', orientation='h', marker=dict(color='#1fb466')))
fig.add_trace(go.Bar(x=[m2.FN_count], y=[m2.model_name], name='FN', orientation='h', marker=dict(color='#dd3f3f')))
fig.add_trace(go.Bar(x=[m2.FP_count], y=[m2.model_name], name='FP', orientation='h', marker=dict(color='#d5a5a5')))
              
fig.update_layout(barmode='stack', title="Outcome Counts",
                  width=600, height=350)
fig.update_xaxes(title_text="Count")
fig.update_yaxes(tickangle=0)
fig.show()

## Recall

In [ ]:
blue_color = '#1f77b4'
orange_color = '#ff7f0e'

In [ ]:
per_class_metrics_df1 = m1.per_class_metrics()
per_class_metrics_df2 = m2.per_class_metrics()

f1_mean = (per_class_metrics_df1["f1"] + per_class_metrics_df2["f1"]) / 2
f1_mean = f1_mean.sort_values()
pr_mean = (per_class_metrics_df1["precision"] + per_class_metrics_df2["precision"]) / 2
pr_mean = pr_mean.sort_values()
rc_mean = (per_class_metrics_df1["recall"] + per_class_metrics_df2["recall"]) / 2
rc_mean = rc_mean.sort_values()

In [ ]:
# Per-class Precision line chart
per_class_metrics_df_sorted1 = per_class_metrics_df1.iloc[f1_mean.index]
per_class_metrics_df_sorted2 = per_class_metrics_df2.iloc[f1_mean.index]

fig = go.Figure()
fig.add_trace(go.Scatter(x=per_class_metrics_df_sorted1["category"], y=per_class_metrics_df_sorted1["precision"],
                            mode='lines', name=m1.model_name, fill='tozeroy', line=dict(width=3)))
fig.add_trace(go.Scatter(x=per_class_metrics_df_sorted2["category"], y=per_class_metrics_df_sorted2["precision"],
                            mode='lines', name=m2.model_name, fill='tozeroy', line=dict(width=3)))

fig.update_layout(title="Per-class Precision", xaxis_title="Category", yaxis_title="Precision",
                    width=None, height=500)
fig.update_xaxes(tickangle=45)
fig.update_yaxes(range=[0., 1.])
fig.show()

In [ ]:
# Per-class Recall line chart
per_class_metrics_df_sorted1 = per_class_metrics_df1.iloc[f1_mean.index]
per_class_metrics_df_sorted2 = per_class_metrics_df2.iloc[f1_mean.index]

fig = go.Figure()
fig.add_trace(go.Scatter(x=per_class_metrics_df_sorted1["category"], y=per_class_metrics_df_sorted1["recall"],
                            mode='lines', name=m1.model_name, fill='tozeroy', line=dict(width=3)))
fig.add_trace(go.Scatter(x=per_class_metrics_df_sorted2["category"], y=per_class_metrics_df_sorted2["recall"],
                            mode='lines', name=m2.model_name, fill='tozeroy', line=dict(width=3)))

fig.update_layout(title="Per-class Recall", xaxis_title="Category", yaxis_title="Recall",
                    width=None, height=500)
fig.update_xaxes(tickangle=45)
fig.update_yaxes(range=[0., 1.])
fig.show()

In [ ]:
# Per-class F1 bar chart
per_class_metrics_df_sorted1 = per_class_metrics_df1.iloc[f1_mean.index]
per_class_metrics_df_sorted2 = per_class_metrics_df2.iloc[f1_mean.index]

fig = go.Figure()
fig.add_trace(go.Scatter(x=per_class_metrics_df_sorted1["category"], y=per_class_metrics_df_sorted1["f1"],
                            mode='lines', name=m1.model_name, fill='tozeroy', line=dict(width=3)))
fig.add_trace(go.Scatter(x=per_class_metrics_df_sorted2["category"], y=per_class_metrics_df_sorted2["f1"],
                            mode='lines', name=m2.model_name, fill='tozeroy', line=dict(width=3)))

fig.update_layout(title="Per-class F1", xaxis_title="Category", yaxis_title="F1",
                    width=None, height=500)
fig.update_xaxes(tickangle=45)
fig.update_yaxes(range=[0., 1.])
fig.show()

## PR-curve

In [ ]:
pr_curve1 = m1.pr_curve()
pr_curve2 = m2.pr_curve()

In [ ]:
import plotly.colors
colors = px.colors.qualitative.Plotly
colors_a = [f"rgba{tuple(list(plotly.colors.hex_to_rgb(c))+[0.35])}" for c in colors]

In [ ]:
# Precision-Recall curve
fig = go.Figure()
fig.add_traces([
    go.Scatter(x=m1.recThrs, y=pr_curve1.mean(-1), name=m1.model_name, line=dict(width=2, color=colors[0]), mode='lines', fill='tozeroy', fillcolor=colors_a[0]),
    go.Scatter(x=m1.recThrs, y=pr_curve1.mean(-1), name=m1.model_name, line=dict(width=2, color=colors[0]), mode='lines', showlegend=False),
    go.Scatter(x=m2.recThrs, y=pr_curve2.mean(-1), name=m2.model_name, line=dict(width=2, color=colors[1]), mode='lines', fill='tozeroy', fillcolor=colors_a[1]),
    go.Scatter(x=m2.recThrs, y=pr_curve2.mean(-1), name=m2.model_name, line=dict(width=2, color=colors[1]), mode='lines', showlegend=False),
])
fig.update_layout(title="Precision-Recall Curve", xaxis_title="Recall", yaxis_title="Precision",
                    width=700, height=600)
fig.update_yaxes(range=[0., 1.01])
fig.update_xaxes(range=[0., 1.])
fig.add_trace(
    go.Scatter(
        x=m1.recThrs,
        y=[1]*len(m1.recThrs),
        name="Perfect",
        line=dict(color='orange', dash='dash'),
        showlegend=True
    )
)
fig.show()

## Calibration

In [ ]:
# Calibration curve (only positive predictions)

fig = go.Figure()

for m_full in model_metrics_full:
    true_probs, pred_probs = m_full.calibration_metrics.calibration_curve()
    fig.add_trace(go.Scatter(x=pred_probs, y=true_probs, mode='lines+markers', name=m_full.model_name))

# Perfectly calibrated line
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Perfectly calibrated', 
                         line=dict(color='orange', dash='dash')))

fig.update_layout(
    title='Calibration Curve (only positive predictions)',
    xaxis_title='Confidence Score',
    yaxis_title='Fraction of True Positives',
    legend=dict(x=0.6, y=0.1),
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1]),
    width=700, height=500
)

fig.show()

In [ ]:
# Confidence score profile

fig = go.Figure()
for m_full in model_metrics_full:
    score_profile = m_full.confidence_score_profile()
    f1_optimal_conf, best_f1 = m_full.get_f1_optimal_conf()
    f1 = score_profile['f1']
    scores = score_profile['scores']

    # downsample
    if len(f1) > 5000:
        f1 = f1[::len(f1)//1000]
        scores = scores[::len(scores)//1000]

    fig.add_trace(go.Scatter(x=scores, y=f1, mode='lines', name=m_full.model_name))

    # Add vertical line for the best threshold
    fig.add_shape(type="line", x0=f1_optimal_conf, x1=f1_optimal_conf, y0=0, y1=best_f1, line=dict(color="gray", width=1, dash="dash"))
    fig.add_annotation(x=f1_optimal_conf, y=best_f1+0.04, text=f"F1-optimal threshold: {f1_optimal_conf:.2f}", showarrow=False)
    
fig.update_layout(
    yaxis=dict(range=[0, 1]),
    xaxis=dict(range=[0, 1], tick0=0, dtick=0.1),
    title='Confidence Score Profile',
    xaxis_title='Confidence Score',
    yaxis_title='F1',
    width=None, height=500,
)

fig.show()

## All Classes

In [ ]:
# Per-class Average Precision (AP)

fig = go.Figure()
for m in model_metrics:
    ap_per_class = m.coco_precision[:, :, :, 0, 2].mean(axis=(0, 1))
    r = np.concatenate([ap_per_class, [ap_per_class[0]]])
    theta = m.cat_names + [m.cat_names[0]]
    fig.add_trace(go.Scatterpolar(r=r, theta=theta, name=m.model_name,
                                  mode='lines+markers',
                                  fill='toself',
                                  hovertemplate='%{theta}: %{r:.2f} (AP)'))

fig.update_layout(
    polar=dict(
        radialaxis=dict(range=[0, 1]),
        angularaxis=dict(direction="clockwise", rotation=90),
        ),
    title="Per-class Average Precision (AP)",
    width=800, height=800,
)

fig.show()